In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from torch import nn
import argparse
import glob
from tensorboardX import SummaryWriter
import pandas as pd

import pathlib
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt

from tqdm import tqdm

import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from utils import data_generator
import numpy as np
import argparse
from tensorboardX import SummaryWriter
from torchsummary import summary
from torch.nn.modules.module import _addindent
from numpy import asarray
from numpy import savetxt

In [2]:
bs = 64

In [3]:


DATASET = 'Earthquakes'
classes = 2



In [4]:
path = pathlib.Path('')

In [5]:
def one_hot_encode(input, labels):
    m = input.shape[0]
    output = np.zeros((m, labels), dtype=int)
    row_index = np.arange(m)
    output[row_index, input] = 1
    return output

def split_xy(data, classes):
    X = data[:, 1:]
    y = data[:, 0].astype(int)
    # hot encode
    #y = one_hot_encode(y, classes)
    return X, y

def create_dataset(X, y, device):
    X_tensor = torch.tensor(X, dtype=torch.float32, device=device)
    y_tensor = torch.tensor(y, dtype=torch.long, device=device)
    return TensorDataset(X_tensor, y_tensor)

def load_data(path, classes):
    data = np.loadtxt(path)
    return split_xy(data, classes)

In [6]:


# load training dataset
X_train, y_train = load_data(path/'Earthquakes_TRAIN.txt', classes) 

# load testing dataset
X_test, y_test = load_data(path/'Earthquakes_TEST.txt', classes)



OSError: Earthquakes_TRAIN.txt not found.

In [ ]:
print('X_train %s   y_train %s' % (X_train.shape, y_train.shape))
print('X_test  %s   y_test  %s' % (X_test.shape, y_test.shape))

In [ ]:
class_0_count = (y_train==0).sum()
class_1_count = (y_train==1).sum()

class_0_count, class_1_count

In [ ]:
cuda = torch.device('cuda')     # Default CUDA device

In [ ]:
train_ds = create_dataset(X_train, y_train, cuda)
test_ds  = create_dataset(X_test, y_test, cuda)

In [ ]:
class_sample_count = [class_0_count, class_1_count] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
weights = 1 / torch.Tensor(class_sample_count)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, bs)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=False)#, sampler = sampler)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=False)

In [ ]:
class BlockLSTM(nn.Module):
    def __init__(self, time_steps, num_layers, lstm_hs, dropout=0.8, attention=False):
        super().__init__()
        self.lstm = nn.LSTM(input_size=time_steps, hidden_size=lstm_hs, num_layers=num_layers)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x):
        # input is of the form (batch_size, num_layers, time_steps), e.g. (128, 1, 512)
        x = torch.transpose(x, 0, 1)
        # lstm layer is of the form (num_layers, batch_size, time_steps)
        x, (h_n, c_n) = self.lstm(x)
        # dropout layer input shape (Sequence Length, Batch Size, Hidden Size * Num Directions)
        y = self.dropout(x)
        # output shape is same as Dropout intput
        return y

In [ ]:
class BlockFCNConv(nn.Module):
    def __init__(self, in_channel=1, out_channel=128, kernel_size=8, momentum=0.99, epsilon=0.001, squeeze=False):
        super().__init__()
        self.conv = nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size)
        self.batch_norm = nn.BatchNorm1d(num_features=out_channel, eps=epsilon, momentum=momentum)
        self.relu = nn.ReLU()
    def forward(self, x):
        # input (batch_size, num_variables, time_steps), e.g. (128, 1, 512)
        x = self.conv(x)
        # input (batch_size, out_channel, L_out)
        x = self.batch_norm(x)
        # same shape as input
        y = self.relu(x)
        return y


In [ ]:
class BlockFCN(nn.Module):
    def __init__(self, time_steps, channels=[1, 128, 256, 128], kernels=[8, 5, 3], mom=0.99, eps=0.001):
        super().__init__()
        self.conv1 = BlockFCNConv(channels[0], channels[1], kernels[0], momentum=mom, epsilon=eps, squeeze=True)
        self.conv2 = BlockFCNConv(channels[1], channels[2], kernels[1], momentum=mom, epsilon=eps, squeeze=True)
        self.conv3 = BlockFCNConv(channels[2], channels[3], kernels[2], momentum=mom, epsilon=eps)
        output_size = time_steps - sum(kernels) + len(kernels)
        self.global_pooling = nn.AvgPool1d(kernel_size=output_size)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # apply Global Average Pooling 1D
        y = self.global_pooling(x)
        return y


In [ ]:
class LSTMFCN(nn.Module):
    def __init__(self, time_steps, num_variables=1, lstm_hs=64, channels=[1, 128, 256, 128]):
        super().__init__()
        self.lstm_block = BlockLSTM(time_steps, 1, lstm_hs)
        self.fcn_block = TCN(time_steps)
        self.dense = nn.Linear(channels[-1] + lstm_hs, num_variables)
        self.softmax = nn.LogSoftmax(dim=1) #nn.Softmax(dim=1)
    def forward(self, x):
        # input is (batch_size, time_steps), it has to be (batch_size, 1, time_steps)
        x = x.unsqueeze(1)
        # pass input through LSTM block
        x1 = self.lstm_block(x)
        x1 = torch.squeeze(x1)
        # pass input through FCN block
        x2 = self.fcn_block(x)
        x2 = torch.squeeze(x2)
        # concatenate blocks output
        x = torch.cat([x1, x2], 1)
        # pass through Linear layer
        x = self.dense(x)
        #x = torch.squeeze(x)
        # pass through Softmax activation
        y = self.softmax(x)
        return y

In [ ]:
time_steps = X_train.shape[1]
num_variables = classes

time_steps, num_variables

In [ ]:
model = LSTMFCN(time_steps, num_variables).cuda()

In [ ]:


# model summary
for m in model.children():
    print("this is mother")
    print(m.training)#, m)
    for j in m.children():
        print(j.training, j)



In [ ]:
[p.shape for p in model.parameters()]

In [ ]:
class SimpleLearner():
    def __init__(self, data, model, loss_func, wd = 1e-5):
        self.data, self.model, self.loss_func = data, model, loss_func
        self.wd = wd
    
    def update_manualgrad(self, x,y,lr):
        y_hat = self.model(x)
        # weight decay
        w2 = 0.
        for p in model.parameters(): w2 += (p**2).sum()
        # add to regular loss
        loss = self.loss_func(y_hat, y) + w2 * self.wd
        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p.sub_(lr * p.grad)
                p.grad.zero_()
        return loss.item()

    def update(self, x,y,lr):
        opt = optim.Adam(self.model.parameters(), lr)
        y_hat = self.model(x)
        loss = self.loss_func(y_hat, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        return loss.item()

    def fit(self, epochs=1, lr=1e-3):
        """Train the model"""
        losses = []
        for i in tqdm(range(epochs)):
            for x,y in self.data[0]:
                current_loss = self.update(x, y , lr)
                losses.append(current_loss)
        return losses
    
    def evaluate(self, X):
        """Evaluate the given data loader on the model and return predictions"""
        result = None
        for x, y in X:
            y_hat = self.model.cpu()(x.cpu().data.numpy())
            result = y_hat if result is None else np.concatenate((result, y_hat), axis=0)
        return result

In [ ]:
model = LSTMFCN(time_steps, num_variables).cuda()

In [ ]:


# depending on the number of classes, use a Binary Cross Entropy or a Negative Log Likelihood loss for more than two classes
loss_func = nn.NLLLoss().cuda() # weight=weights



In [ ]:

from torch import optim
lr = 2e-2
learner = SimpleLearner([train_dl, test_dl], model, loss_func)
losses = learner.fit(10)



In [ ]:
plt.plot(losses)

In [ ]:
test_dl

In [ ]:
y_pred = learner.evaluate(test_dl).cpu().data.numpy()

In [ ]:
model = LSTMFCN(time_steps, num_variables).cuda()

In [ ]:
from fastai.basics import *
from fastai.basics import *
data = DataBunch(train_dl=train_dl, valid_dl=test_dl)
learner = Learner(data, model, loss_func=loss_func, metrics=accuracy)